I often encounter code that looks like this:

```
with get_database_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM mytable")
```

or 

```
with open('config.json', 'w') as f:
    contents = json.load(f)
```

and didn't really know what it means. Why do we use `with...`-statements? What is the problem with just calling `contents = json.load('config.json')`?

    

## The problem of opening but not closing

The main motivation of using a combination of contextmanagers and with-statements has to do with _resource management_. We can use python to open files, connect to databases, start client processes. But everytime we open, connect, or start something, we need to close it after we're done. Otherwise we take up too many resources.

Imagine you would open 50 instances of GOOGLE CHROME. Your Laptop would explode. To prevent using up too many of your Laptop's resources, you _close_ instances of GOOGLE CHROME that you don't longer need.

So we have to remember to always call `file.close()` or `connection.close()`. But it's not enough to _remember_ calling a `.close()` method. We have to make sure it is executed. That's what the combination of contextmanager & with-statement ensures.

The following code is problematic, because if there's an error in `do_something()`, the `f.close()` will not be triggered. The file will not be closed and resources are not cleared up.

In [16]:
def do_something_that_has_a_bug(f):
    raise ValueError

f = open('../.gitignore', 'r')
do_something_that_has_a_bug(f)
f.close()

ValueError: 

In [17]:
# There was a bug before calling f.close(), so the file didn't close!
f.closed

False

To fix this, we use the combination of contextmanager & with-statements:

In [18]:
with open('../.gitignore', 'r') as f:
    do_something_that_has_a_bug(f)

ValueError: 

In [19]:
# Even though there was a bug, the file was closed
f.closed

True

## Custom contextmanager functions

Often I have to (1) connect to a database, (2) run a query to fetch data, and (3) close the connection.

In [22]:
from contextlib import contextmanager


@contextmanager
def get_connection():
    conn = psycopg2.connect(
        host='localhost',
        port='15432',
        dbname='my_database',
        user='',
        password=''
    )
    try:
        yield conn
    finally:
        conn.close()

In [23]:
with get_connection() as conn:
    do_something_that_has_a_bug(conn)

NameError: name 'psycopg2' is not defined

In [ ]:
conn.closed # True